In [1]:
!pip install faiss-cpu==1.8.0 -q
!pip install langchain==0.2.5 -q
!pip install langchain-community==0.2.5 -q
!pip install langchain-core===0.2.7 -q
!pip install langchain-openai==0.0.5 -q
!pip install langchain-text-splitters==0.2.1 -q
!pip install langsmith==0.1.81 -q
!pip install openai==1.10.0 -q
!pip install tiktoken==0.7.0 -q
!pip install pdfminer.six -q
!pip install langchain-google-genai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.2.2 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.2.7 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [7]:
!pip install pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.4 MB/s eta 0:00:00


In [2]:
import sys
import os
import numpy as np
import pandas as pd
import time
import pickle
from langchain_community.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain_community.vectorstores import FAISS

from langchain.chains import  ConversationalRetrievalChain ,RetrievalQA
from langchain_community.llms import OpenAI
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate
from langchain.schema import HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from google.colab import userdata
from langchain.chains.question_answering import load_qa_chain

#import warnings
#warnings.filterwarnings("ignore")

In [3]:
# Veri Setleri aşağıdadır.
# https://cbddo.gov.tr/arastirma-raporlari/
# https://cbddo.gov.tr/SharedFolderServer/Genel/File/TR-UlusalYZStratejisi2021-2025.pdf

In [5]:
# Çalışma ortamında bir klasör oluştur !
kaynak = 'belgeler'
# Klasör içerideki dokumanları göster
dokuman_listesi = os.listdir(kaynak)
print(dokuman_listesi)


['21.Chatbot-Uygulaması-ve-ChatGPT-Örneği-Değerlendirme-Raporu.pdf', 'TR-UlusalYZStratejisi2021-2025.pdf']


In [29]:
bolunmus_dokuman = []

for i in dokuman_listesi:
    loader = PyPDFLoader("belgeler/"+i)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200 , length_function = len)
    docs = text_splitter.split_documents(data)
    for k in range(len(docs)):
        bolunmus_dokuman.append(docs[k])
print("PyPDFLoader ",len(bolunmus_dokuman))

bolunmus_dokuman1 = []

for i in dokuman_listesi:
    loader = PDFMinerLoader("belgeler/"+i)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200 , length_function = len)
    docs = text_splitter.split_documents(data)
    for k in range(len(docs)):
        bolunmus_dokuman1.append(docs[k])
print("PDFMinerLoader ",len(bolunmus_dokuman1))

PyPDFLoader  264
PDFMinerLoader  209


In [40]:
#PyPDFLoader
bolunmus_dokuman[22]

Document(metadata={'source': 'belgeler/21.Chatbot-Uygulaması-ve-ChatGPT-Örneği-Değerlendirme-Raporu.pdf', 'page': 13}, page_content='14 \n \n\uf0b7 Önceden tanımlanmış yapıları kullanmadıkları i çin, yapay zekâ c hatbot’larıyla \nyapılan sohbet, k ural tabanlı chatbot’larla  yapılana kıyasla daha az tahmin \nedilebilirdir.13 \n \nŞekil 3: Yapay Zekâ Chatbot’ları Nasıl Çalışır?  \n \nKaynak:  Mindtitan - Chatbot: The Ultimate Overview, https://mindtitan.com/resources/guides/chatbot/ ,  \nErişim Tarihi: 17.02.2023.  \n \nMakine öğrenmesi algoritmalarının kullanımı sayesinde, artık yeni kalıp eşleştirme \nkurallarının manuel  olarak tanımlanmas ına ve kodlanmasına gerek kalmamakta , bu da \nchatbot’ların  daha esnek olmasını ve artık alana özgü b ilgiye bağımlılığı ortadan \nkaldırmaktadır.  Yapay zekâ  modelleri; bilgi erişimine dayalı modeller  (information retrieval \nbased models ) ve ü retken modeller  (generative models ) olarak ikiye ayrılmaktadır.  \n\uf0b7 Bilgi Erişimine Dayalı 

In [39]:
#PDFMinerLoader
bolunmus_dokuman1

[Document(metadata={'source': 'belgeler/21.Chatbot-Uygulaması-ve-ChatGPT-Örneği-Değerlendirme-Raporu.pdf'}, page_content='1 \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\x0cİÇİNDEKİLER \n\nŞEKİLLER. .......................................................................................................................................4 \n\nTABLOLAR ......................................................................................................................................4 \n\nGRAFİKLER.....................................................................................................................................4 \n\nBİRİNCİ BÖLÜM \n\nCHATBOT UYGULAMASI \n\n1.1.  CHATBOT NEDİR? ...............................................................................................................6 \n\n1.2.  CHATBOT UYGULAMALARININ TARİHÇESİ ......................................